In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import os

In [2]:
# Load the data

path = ("C:/kaggle/pets/PetImages")

path_cat = path + "/Cat"
path_dog = path + "/Dog"

num_img = 500

size = 256

def load_dataset():
    X = []
    y = []
    for i in range(num_img):
        try:
            img = Image.open(os.path.join(path_cat, f"{i}.jpg")).resize((size, size))
            img = img.convert('RGB')  
            X.append(np.array(img))
            y.append(1)
        except Exception as e:
            print(f"Error loading image {i} from Cat: {e}")
    for i in range(num_img):
        try:
            img = Image.open(os.path.join(path_dog, f"{i}.jpg")).resize((size, size))
            img = img.convert('RGB')  
            X.append(np.array(img))
            y.append(0)
        except Exception as e:
            print(f"Error loading image {i} from Dog: {e}")
    return np.array(X), np.array(y)

In [16]:
X, y = load_dataset()

In [17]:
X_flatten = X.reshape(X.shape[0], -1).T
X_flatten = X_flatten / 255.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_flatten.T, y, test_size=0.2, random_state=42)

In [19]:
X_train = X_train.T
X_test = X_test.T


In [20]:
y_train = y_train.reshape(1, -1)
y_test = y_test.reshape(1, -1)

In [21]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [22]:
def initialize_with_zeros(dim):
    w = np.zeros((dim, 1))
    b = 0
    return w, b

In [23]:
def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)
    cost = -1/m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    dw = 1/m * np.dot(X, (A - Y).T)
    db = 1/m * np.sum(A - Y)
    return dw, db, cost

In [24]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=False):
    costs = []
    for i in range(num_iterations):
        dw, db, cost = propagate(w, b, X, Y)
        w = w - learning_rate * dw
        b = b - learning_rate * db
        if i % 100 == 0:
            costs.append(cost)
            if print_cost:
                print(f"Cost after iteration {i}: {cost}")
    return w, b, costs

In [25]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    A = sigmoid(np.dot(w.T, X) + b)
    for i in range(A.shape[1]):
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
    return Y_prediction

In [26]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    w, b = initialize_with_zeros(X_train.shape[0])
    w, b, dw, db, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    print(f"Train accuracy: {100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100}")
    print(f"Test accuracy: {100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100}")
    return w, b

In [27]:
w, b = model(X_train, y_train, X_test, y_test, num_iterations=2000, learning_rate=0.005, print_cost=True)

Cost after iteration 0: 0.6931471805599452


C:\Users\KATANA\AppData\Local\Temp\ipykernel_15016\3348064.py:4: RuntimeWarning: divide by zero encountered in log
  cost = -1/m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
C:\Users\KATANA\AppData\Local\Temp\ipykernel_15016\3348064.py:4: RuntimeWarning: invalid value encountered in multiply
  cost = -1/m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))


Cost after iteration 100: 12.275031162350128
Cost after iteration 200: 20.051281669001593
Cost after iteration 300: 19.711447048199904
Cost after iteration 400: 18.32093948531712
Cost after iteration 500: nan
Cost after iteration 600: nan


KeyboardInterrupt: 